In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, GRU, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D
from keras.layers import Input
import pickle
from keras.utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split
import datetime
from keras.models import load_model


2022-11-24 19:49:54.140748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 19:49:54.310348: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-24 19:49:54.903325: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:
2022-11-24 19:49:54.903417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
MODEL = "CharLSTM+ConjoinedWords"
SPECIALITY = "emed"
DATA_PATH = "data/EM_ED_240222/"
DATA_VERSION = 'EM_ED_240222'
file_path = 'best_model_'+MODEL+'_'+SPECIALITY+'_'+DATA_VERSION+'_'+str(datetime.datetime.now())

MIN_WORD_FREQUENCY = 5
max_len = 50

In [4]:
max_len = 100
max_len_char = 4

In [5]:
train_sentences = open(DATA_PATH+SPECIALITY+'_train.tokens',"r").readlines()[:10000]
len(train_sentences)

10000

In [6]:
val_sentences = open(DATA_PATH+SPECIALITY+'_val.tokens',"r").readlines()[:10000]

In [7]:
import math

def process_sentences(sentences, n = 4):
    out_sentences = []
    out_tags = []
    for sentence in sentences:
        x = []
        y = []
        stripped_sentence = sentence.strip().lower().replace(" ", "")
        for shingle in [stripped_sentence[k:k+n] for k in range(len(stripped_sentence)-n+1)]:
            x.append(shingle)
            if shingle in sentence.lower():
                y.append(0)
            else:
                y.append(1)
        out_sentences.append(x)
        out_tags.append(y)
    return out_sentences, out_tags

In [8]:
# import math

# def process_sentences(sentences, n = 4):
#     out_sentences = []
#     out_tags = []
#     for i in range(len(sentences)):
#         try:
#             sentence = sentences[i].strip().lower()
#             x = []
#             tags = []
#             head = 0
#             while head < len(sentence)-n+1:
#                 curr_shingle = ''
#                 pointer = head
#                 tag = 0
#                 while len(curr_shingle) < n:
#                     if pointer >= len(sentence):
#                         curr_shingle+= "X"*(n-len(curr_shingle))
#                         break
#                     elif sentence[pointer]!=' ':
#                         curr_shingle += sentence[pointer]
#                     else:
#                         tag+=1
#                     pointer += 1
#                 head += math.ceil((tag+1)/2)
#                 if sentence[head] == ' ':
#                     head+=1
#                 x.append(curr_shingle)
#                 tags.append(tag)
#             out_sentences.append(x)
#             out_tags.append(tags)
#         except:
#             print(sentence,x,tags)
#     return out_sentences, out_tags

In [9]:
a,b = process_sentences(['abscess of a b left knee', 'facility id:'], n =3)

a

[['abs',
  'bsc',
  'sce',
  'ces',
  'ess',
  'sso',
  'sof',
  'ofa',
  'fab',
  'abl',
  'ble',
  'lef',
  'eft',
  'ftk',
  'tkn',
  'kne',
  'nee'],
 ['fac', 'aci', 'cil', 'ili', 'lit', 'ity', 'tyi', 'yid', 'id:']]

In [10]:
train_sentences, train_tags= process_sentences(train_sentences,max_len_char)
val_sentences, val_tags= process_sentences(val_sentences,max_len_char)

In [11]:
word_count_dict = {}

for i in train_sentences:
    for j in i:
        if j.lower().strip() not in word_count_dict:
            word_count_dict[j.lower().strip()] = 1
        else:
            word_count_dict[j.lower().strip()] += 1

In [12]:
MIN_WORD_FREQUENCY = 2

In [13]:
word2id = {}
word_count = 0

for i in train_sentences:
    for j in i:
        if j.lower().strip() not in word2id and word_count_dict[j.lower().strip()] >= MIN_WORD_FREQUENCY:
            word2id[j.lower().strip()] = word_count
            word_count += 1

In [14]:
word2id['<UNK>'] = len(word2id)
word2id['<PAD>'] = len(word2id)

In [15]:
import json
pickle.dump(word2id,open('meta/'+file_path+'_word2id.pkl','wb'))

with open('meta/'+file_path+'_word2id.json', 'w') as outfile:
    json.dump(word2id, outfile)

In [16]:
tag2id = {}
tag_count = 0

for i in train_tags:
    for j in i:
        if j not in tag2id:
            tag2id[j] = tag_count
            tag_count += 1

In [17]:
tag2id['<PAD>'] = len(tag2id)

In [18]:
pickle.dump(tag2id,open('meta/'+file_path+'_tag2id.pkl','wb'))

with open('meta/'+file_path+'_tag2id.json', 'w') as outfile:
    json.dump(tag2id, outfile)

In [19]:
X = []

for s in train_sentences:
    tmp = []
    for w in s:
        if w.lower().strip() in word2id:
            tmp.append(word2id[w.lower().strip()])
        else:
            tmp.append(word2id['<UNK>'])
    X.append(tmp)

In [20]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", truncating="post", value=word2id['<PAD>'])

In [21]:
y = [[tag2id[w] for w in s] for s in train_tags]

In [22]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", truncating="post", value=tag2id["<PAD>"])

## Preparing Validation Data

In [23]:
X_val = []

for s in val_sentences:
    tmp = []
    for w in s:
        if w.lower().strip() in word2id:
            tmp.append(word2id[w.lower().strip()])
        else:
            tmp.append(word2id['<UNK>'])
    X_val.append(tmp)

In [24]:
X_val = pad_sequences(maxlen=max_len, sequences=X_val, padding="post",truncating="post", value = word2id['<PAD>'])

In [25]:
y_val = []

for s in val_tags:
    tmp = []
    for w in s:
        if w in tag2id:
            tmp.append(tag2id[w])
        else:
            tmp.append(tag2id['<PAD>'])
    y_val.append(tmp)

In [26]:
y_val = pad_sequences(maxlen=max_len, sequences=y_val, padding="post",truncating="post", value=tag2id["<PAD>"])

## Char Features

In [27]:
chars = set([w_i for w in word2id.keys() for w_i in w])
n_chars = len(chars)
print(sorted(chars))

['"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'D', 'K', 'N', 'P', 'U', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', 'ﾀ', 'ﾂ', 'ﾠ', 'ﾢ', 'ﾰ', 'ﾷ', 'ﾾ', '\uffbf', 'ￂ', '￢', '\uffef']


In [28]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["<UNK>"] = 1
char2idx["<PAD>"] = 0

In [29]:
pickle.dump(char2idx,open('meta/'+file_path+'_char2id.pkl','wb'))

with open('meta/'+file_path+'_char2id.json', 'w') as outfile:
    json.dump(char2idx, outfile)

## Evaluating on Test Dataset

In [30]:
test_sentences = open(DATA_PATH+SPECIALITY+'_test.tokens',"r").readlines()[:10000]

print(len(test_sentences))
test_sentences , test_tags = process_sentences(test_sentences, n= max_len_char)
print(len(test_sentences), len(test_tags))

10000
10000 10000


In [31]:
#Load a previous trained model

# word2id = pickle.load(open('../entity_tagger/meta/best_model_CharLSTM+ConjoinedWords_EM-ER_EM_ER_140420_2021-04-16 10:39:45.603327_word2id.pkl','rb'))
# tag2id = pickle.load(open('../entity_tagger/meta/best_model_CharLSTM+ConjoinedWords_EM-ER_EM_ER_140420_2021-04-16 10:39:45.603327_tag2id.pkl','rb'))
# char2idx = pickle.load(open('../entity_tagger/meta/best_model_CharLSTM+ConjoinedWords_EM-ER_EM_ER_140420_2021-04-16 10:39:45.603327_char2id.pkl','rb'))

In [32]:
X_test = []

for s in test_sentences:
    tmp = []
    for w in s:
        if w.lower().strip() in word2id:
            tmp.append(word2id[w.lower().strip()])
        else:
            tmp.append(word2id['<UNK>'])
    X_test.append(tmp)

In [33]:
X_test = pad_sequences(maxlen=max_len, sequences=X_test, padding="post",truncating="post", value = word2id['<PAD>'])

In [34]:
y_test = []

for s in test_tags:
    tmp = []
    for w in s:
        if w in tag2id:
            tmp.append(tag2id[w])
        else:
            tmp.append(tag2id['<PAD>'])
    y_test.append(tmp)

In [35]:
y_test = pad_sequences(maxlen=max_len, sequences=y_test, padding="post",truncating="post", value=tag2id["<PAD>"])

In [36]:
y_test = [to_categorical(i, num_classes=len(tag2id)) for i in y_test]

In [37]:
type(X)

numpy.ndarray

In [38]:
test_X_char = []

for sentence in test_sentences:
    sent_seq = []
    words = sentence
    for i in range(max_len):
        try:
            w = words[i].lower()            
        except:
            sent_seq.append([char2idx.get("<PAD>")]*max_len_char)
            continue
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(w[j]))
            except:
                word_seq.append(char2idx.get("<PAD>"))
        sent_seq.append(word_seq)
    test_X_char.append(np.array(sent_seq))

In [39]:
class TestCallback(keras.callbacks.Callback):
    def __init__(self, X,y):
        self.X = X
        self.y = y

    def on_epoch_end(self, epoch, logs):
        loss, acc = self.model.evaluate(self.X,self.y)
        logs['test_loss'] = loss
        logs['test_acc'] = acc
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [40]:
#word_model = Sequential()
#word_model.add(Embedding(input_dim=len(word2id)+1, output_dim=30, input_length=max_len))

#char_model = Sequential()
#char_model.add(TimeDistributed(Embedding(input_dim=len(char2idx)+1, output_dim=10, input_length=max_len_char)))
#char_model.add(TimeDistributed(LSTM(units=20, return_sequences=False, recurrent_dropout=0.2)))

#final_model = Sequential()
#final_model.add(Concatenate([word_model, char_model]))
#final_model.add(SpatialDropout1D(0.2))
#final_model.add(Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.2)))
#final_model.add(TimeDistributed(Dense(len(tag2id), activation="softmax")))

#final_model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=['acc'])
#keras.utils.plot_model(final_model, "multi_input_and_output_model-2.png", show_shapes=True)

In [41]:
#from keras.models import Functional

In [42]:
#history = final_model.fit(x=[X_train, X_char_train], y=y_train, epochs=1, batch_size=128, verbose=1)
#history = final_model.fit(training_generator,
#                    validation_data=validation_generator, 
#                    epochs=1,
#                    verbose=1, callbacks = callbacks, use_multiprocessing=False) 

In [43]:
#from keras_contrib.layers import CRF

# input and embedding for words
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=len(word2id)+1, output_dim=30,
                     input_length=max_len)(word_in)

# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=len(char2idx)+1, output_dim=10,
                           input_length=max_len_char))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.2))(emb_char)

# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.2)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.2))(x)

out = TimeDistributed(Dense(len(tag2id), activation="softmax"))(main_lstm) # a dense layer as suggested by neuralNer



# crf = CRF(len(tag2id))

# out = crf(model)

2022-11-24 19:49:59.790082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 19:49:59.825711: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 19:49:59.825923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 19:49:59.826338: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [44]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, 
    min_lr=0.0001)

tb = keras.callbacks.TensorBoard(log_dir='./logs')

model_checkpoint = keras.callbacks.ModelCheckpoint(filepath='models/'+file_path+'.hdf5', monitor='val_loss', 
    save_best_only=True)

In [45]:
#from keras_contrib.losses import crf_loss
#from keras_contrib.metrics import crf_accuracy

model = Model([word_in, char_in], out)

model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=['acc'])

#model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy])
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
print(model.summary())

model_json = model.to_json()
with open("configs/"+file_path+".json", "w") as json_file:
    json_file.write(model_json)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100, 4)]     0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 time_distributed (TimeDistribu  (None, 100, 4, 10)  810         ['input_2[0][0]']                
 ted)                                                                                             
                                                                                                  
 embedding (Embedding)          (None, 100, 30)      849300      ['input_1[0][0]']            

In [55]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, X, sentences,Y,tag_count, batch_size=32,shuffle=True):
        'Initialization'
        self.X = X
        self.tag_count = tag_count
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.Y = Y
        self.curr_index = 0
        self.on_epoch_end()
        self.sentences = np.array(sentences, dtype=object)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.X)/self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]        
        
        X_char = []
        
        # for every sentence
        for sentence in self.sentences[indexes]:
            sent_seq = []
            # get the words in the sentence
            words = sentence
            # iterate from 1 to max_len
            for i in range(max_len):
                # check if we have a word at i-th postion
                try:
                    w = words[i].lower()            
                except:
                    # word is not there, pad it.
                    sent_seq.append([char2idx.get("<PAD>")]*max_len_char)
                    continue
                
                # iterate over 1 to 4
                word_seq = []
                for j in range(max_len_char):
                    # if j-th letter is known, get the index value.
                    try:
                        word_seq.append(char2idx.get(w[j]))
                    except:
                        # for unknown letters, use the padding.
                        word_seq.append(char2idx.get("<PAD>"))
                        
                # add the word seq to the sent seq
                sent_seq.append(word_seq)
            # construct the index array for the sentence (aka X[i])    
            X_char.append(np.array(sent_seq))
        
        # collect all the arrays
        X_char = np.array(X_char)
        
        
        # construct the one-hot embedding.
        y_tmp = self.Y[indexes]
        y_tmp = [to_categorical(i, num_classes=self.tag_count) for i in y_tmp]
        y_tmp = np.array(y_tmp)
        
        #print("\nindexes[1]", indexes[1])
        #print("self.X", self.X[indexes].shape)
        
        #print("X_char", X_char.shape)
        #print("y_tmp", np.array(y_tmp).shape)
        
        X_char = np.asarray(X_char).astype(np.float32)
        y_tmp = np.asarray(y_tmp).astype(np.float32)
        
        #print(" ", index)
        return [np.asarray(self.X[indexes]).astype(np.float32), X_char], y_tmp

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [56]:
def encode_sentence_chars(sents, c_index, max_len):
    X_char = []

    # for every sentence
    for sentence in sents:
        sent_seq = []
        # get the words in the sentence
        words = sentence
        # iterate from 1 to max_len
        for i in range(max_len):
            # check if we have a word at i-th postion
            try:
                w = words[i].lower()            
            except:
                # word is not there, pad it.
                sent_seq.append([c_index.get("<PAD>")]*max_len_char)
                continue

            # iterate over 1 to 4
            word_seq = []
            for j in range(max_len_char):
                # if j-th letter is known, get the index value.
                try:
                    word_seq.append(c_index.get(w[j]))
                except:
                    # for unknown letters, use the padding.
                    word_seq.append(c_index.get("<PAD>"))

            # add the word seq to the sent seq
            sent_seq.append(np.array(word_seq))
        # construct the index array for the sentence (aka X[i])    
        X_char.append(np.array(sent_seq))

    # collect all the arrays
    return np.array(X_char)

In [57]:
#y_train = np.array([to_categorical(i, num_classes=len(tag2id)) for i in y])
#X_train = X
#X_train = np.asarray(X_train).astype(np.float32)
#X_char_train = encode_sentence_chars(train_sentences, char2idx, max_len)
#X_char_train = np.asarray(X_char_train).astype(np.float32)

In [58]:
#[X_train, X_char_train]

In [59]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 100) <dtype: 'float32'>
(None, 100, 4) <dtype: 'float32'>
(None, 100, 3) <dtype: 'float32'>
input_2 [(None, 100, 4)] float32
input_1 [(None, 100)] float32
time_distributed (None, 100, 4) float32
embedding (None, 100) float32
time_distributed_1 (None, 100, 4, 10) float32
concatenate [(None, 100, 30), (None, 100, 20)] float32
spatial_dropout1d (None, 100, 50) float32
bidirectional (None, 100, 50) float32
time_distributed_2 (None, 100, 100) float32


[None, None, None, None, None, None, None, None, None]

In [60]:
#callbacks = [reduce_lr,model_checkpoint,TestCallback(X_test, np.array(y_test)),tb]

callbacks = [reduce_lr,model_checkpoint,tb]

In [61]:
#history = model.fit(x=[np.array(X_train), np.array(X_char_train)], y=y_train, epochs=1, batch_size=128, verbose=1)

In [62]:
training_generator = DataGenerator(X, train_sentences, y,len(tag2id), batch_size=1024) #3000)
validation_generator = DataGenerator(X_val, val_sentences, y_val,len(tag2id), batch_size=1024) #3000)

In [ ]:
history = model.fit(training_generator,
                    validation_data=validation_generator, 
                    epochs=10,
                    verbose=1, callbacks = callbacks, use_multiprocessing=False) 

Epoch 1/10
9/9 [==============================] - 10s 761ms/step - loss: 0.1101 - acc: 0.9719 - val_loss: 0.0975 - val_acc: 0.9715 - lr: 0.0010
Epoch 2/10
9/9 [==============================] - 6s 671ms/step - loss: 0.0879 - acc: 0.9786 - val_loss: 0.0785 - val_acc: 0.9784 - lr: 0.0010
Epoch 3/10
9/9 [==============================] - 6s 670ms/step - loss: 0.0699 - acc: 0.9837 - val_loss: 0.0691 - val_acc: 0.9790 - lr: 0.0010
Epoch 4/10
9/9 [==============================] - 6s 668ms/step - loss: 0.0563 - acc: 0.9866 - val_loss: 0.0564 - val_acc: 0.9832 - lr: 0.0010
Epoch 5/10
9/9 [==============================] - ETA: 0s - loss: 0.0466 - acc: 0.9883

In [65]:
model.save_weights("unjoiner.hdf5")

In [67]:
file_path

'best_model_CharLSTM+ConjoinedWords_emed_EM_ED_240222_2022-11-24 19:49:56.146535'

## Loading Best Model to Test

In [68]:
from keras.models import load_model

#custom_objects={'CRF': CRF,'crf_loss':crf.loss_function,'crf_viterbi_accuracy':crf.accuracy}#,'crf_marginal_accuracy':crf_marginal_accuracy}

model = load_model('models/'+file_path+'.hdf5')#,custom_objects=custom_objects)

test_pred = model.predict([np.asarray(X_test).astype(np.float32), np.asarray(test_X_char).astype(np.float32)], verbose=1, batch_size=1024)

10/10 [==============================] - 1s 67ms/step


In [69]:
idx2tag = {i: w for w, i in tag2id.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out

In [70]:
modified_test_pred = []

In [71]:
for i in range(len(test_pred)):
    modified_test_pred.append(test_pred[i][:len(test_sentences[i])])

In [72]:
modified_test_true = []

In [73]:
for i in range(len(y_test)):
    modified_test_true.append(y_test[i][:len(test_sentences[i])])

In [74]:
pred = pred2label(modified_test_pred)
true = pred2label(modified_test_true)

In [75]:
flag = False
for i,k in enumerate(zip(test_sentences[54], pred[54])):
    if k[1] == 1 and not Flag:
        print(" ",end='')
        Flag = True
    elif k[1] == 0:
        Flag = False    
    if i == 0:
        print(k[0],end='')        
    else:
        print(k[0][3],end='')

gastrointestinal noabdominal pain,nodiarrhea ,nonausea andno

In [76]:
for i,k in enumerate(zip(test_sentences[54], pred[54])):
    print(k)

('gast', 0)
('astr', 0)
('stro', 0)
('troi', 0)
('roin', 0)
('oint', 0)
('inte', 0)
('ntes', 0)
('test', 0)
('esti', 0)
('stin', 0)
('tina', 0)
('inal', 0)
('naln', 1)
('alno', 1)
('lnoa', 1)
('noab', 1)
('oabd', 1)
('abdo', 0)
('bdom', 0)
('domi', 0)
('omin', 0)
('mina', 0)
('inal', 0)
('nalp', 1)
('alpa', 1)
('lpai', 1)
('pain', 1)
('ain,', 1)
('in,n', 1)
('n,no', 1)
(',nod', 1)
('nodi', 1)
('odia', 1)
('diar', 0)
('iarr', 0)
('arrh', 0)
('rrhe', 0)
('rhea', 0)
('hea,', 1)
('ea,n', 1)
('a,no', 1)
(',non', 1)
('nona', 1)
('onau', 1)
('naus', 0)
('ause', 0)
('usea', 0)
('seaa', 1)
('eaan', 1)
('aand', 1)
('andn', 1)
('ndno', 1)


In [77]:
with open('data/'+file_path+'_list_true.txt','w') as f:
    for i in true:
        for j in i:
            f.write(str(j)+" ")
        f.write("\n")

In [78]:
with open('data/'+file_path+'_list_pred.txt','w') as f:
    for i in pred:
        for j in i:
            f.write(str(j)+" ")
        f.write("\n")

In [79]:
with open('data/'+file_path+'_list_shingles.txt','w') as f:
    for i in test_sentences:
        for j in i:
            f.write(str(j)+" ")
        f.write("\n")

In [80]:
test_sentences = open(DATA_PATH+SPECIALITY+'_test.tokens',"r").readlines()[:10000]

with open('data/'+file_path+'_sentences.txt','w') as f:
    for i in test_sentences:
        for j in i.split(" "):
            f.write(str(j)+" ")
        f.write("\n")

## Classification Report

In [81]:
import os
from sklearn.metrics import f1_score, classification_report
y_true = []
y_pred = []
pred = open('data/'+file_path+'_list_pred.txt',"r")

with open('data/'+file_path+'_list_true.txt',"r") as f:
    for line in f:
        line = line.split()
        labels = pred.readline().split()
        for i in range(len(line)):
            if i < len(labels):
                y_pred+=[labels[i]]
            else:
                y_pred+=["o"]
            y_true+=[line[i]]

In [82]:
report = classification_report(y_true, y_pred,output_dict=True)

/home/sudarsun/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sudarsun/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sudarsun/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [83]:
df = pd.DataFrame(report).transpose()

In [84]:
df.to_csv('reports/'+file_path+'_results.csv')

In [85]:
df

,precision,recall,f1-score,support
0,0.910071,0.869025,0.889075,145051.00000
1,0.894046,0.926196,0.909837,172971.00000
<PAD>,0.000000,0.000000,0.000000,0.00000
accuracy,0.900120,0.900120,0.900120,0.90012
macro avg,0.601372,0.598407,0.599637,318022.00000
weighted avg,0.901355,0.900120,0.900367,318022.00000


## Confusion Matrix

In [86]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [87]:
cm_df = pd.DataFrame(confusion_matrix(y_true,y_pred))

In [88]:
import pandas as pd

In [89]:
cm_df.rows = sorted(unique_labels(y_true, y_pred))
cm_df.columns = sorted(unique_labels(y_true, y_pred))

/tmp/ipykernel_899637/3738547203.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  cm_df.rows = sorted(unique_labels(y_true, y_pred))


In [90]:
cm_df.to_csv('reports/'+file_path+'_confusion_matrix.csv')

## Tag level analysis

In [91]:
idx2word = {i: w for w, i in word2id.items()}

In [95]:
test_sentences = open(DATA_PATH+SPECIALITY+'_test.tokens',"r").readlines()

In [97]:
count = 0
errors = []

for i in range(len(X_test)):
    words = test_sentences[i].strip().split(' ')[:50]
    try:
        idx = words.index('<PAD>')
    except:
        idx = len(words)
    predicted = modified_test_pred[i].argmax(axis=1)
    true = modified_test_true[i].argmax(axis=1)
    for j in range(idx):
        if predicted[j] != true[j]:
            count+=1
            error = np.zeros((4,),dtype=object)
            str1 = ' '.join(str(e) for e in words[:idx])
            error[0] = (str1)
            error[1] = (words[j])
            error[2] = (idx2tag[true[j]])
            error[3] = (idx2tag[predicted[j]])
            errors.append(error)
print (count)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
errors_df = pd.DataFrame(errors)
errors_df.columns = ['Sentence','Word', 'Ground Truth', 'Predicted']

In [ ]:
from pandas import ExcelWriter

for i in range(0,len(errors_df),100000):
    writer = ExcelWriter('reports/'+file_path+'_test_errors_keywords'+str(i)+'.xlsx')
    errors_df[i:min(i+100000,len(errors_df))].to_excel(writer,index=False)
    writer.save()